In [1]:
import requests, bs4, time
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
from datetime import date
import os
import re
import scraping_funcs_harnham as sf
import pickle


In [2]:
url="https://glossarytech.com/"
req=Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage=urlopen(req).read()
page_soup = soup(webpage, "html.parser")

In [3]:
# scrape categories and clean words, to match urls

categories = page_soup.find_all('a', href=re.compile('^/terms/'))
categories_list = [str(categories[x]).split('"')[1] for x in range(len(categories))]
categories_list = [str(categories_list[x]).split('terms/')[1] for x in range(len(categories_list))]


In [4]:
categories_list

['front_end-technologies',
 'javascript',
 'php',
 'ruby',
 'java',
 'c-net',
 'python',
 'c_plus_plus',
 'c',
 'scala',
 'golang',
 'functional-programming-fp',
 'other-programming_languages',
 'ios',
 'android',
 'cross_platform-hybrid',
 'databases',
 'quality_assurance-qa',
 'game_development',
 'software-infrastructure-devops',
 'web_design',
 'product_management',
 'development_methodologies',
 'software_architecture',
 'fundamental_programming_concepts',
 'programming_paradigms',
 'data-science',
 'tools',
 'roles_in_software_development',
 'embedded',
 'cloud_computing',
 'cyber_security',
 'general_terms',
 'cyber_security/arcsight',
 'cyber_security/nessus',
 'cyber_security/iso']

In [5]:
# scrape tags terms and headline terms
### create category dictionary
tag = []
tec = []
tech_dict ={}
for i in categories_list:    
    tech_dict[i] =""
    for j in ["","/page2","/page3","/page4","/page5","/page6","/page7","/page8"]:
        try:
            url = "https://glossarytech.com/terms/"+i+j
            req=Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage=urlopen(req).read()
            page_soup = soup(webpage, "html.parser")

            tags = page_soup.find_all('a', href="/terms/go_to_term/")
            b = [x.text for x in tags]
            tag+=b
            tech_dict[i]+str(b)
            cells = page_soup.find_all('tr',{'data-term' : True})
            for element in cells:
                a = [x.text for x in element.find_all('a',{'href':re.compile('^/terms/')})]    
                tec.append(a)
                tech_dict[i] += str(a)
            
        except:
            pass

In [6]:
# make tec into a flat lists rather than list of lists
new_tech_list = [item for sublist in tec for item in sublist]

In [7]:
# join the lists
all_tech = new_tech_list + tag

In [8]:
# take only unique values
all_tech = [x.lstrip().rstrip() for x in all_tech]
all_tech_dict = list(dict.fromkeys(all_tech))

In [9]:
# convert to dataframe for pickling
all_tech_df = pd.DataFrame(all_tech_dict)
all_tech_df

,0
0,Front-end
1,HTML5
2,CSS
3,JavaScript
4,AJAX
...,...
1847,Virtual
1848,Visual Programming Language
1849,VOD
1850,W3C


In [10]:
all_tech_df.columns = ['tech']

In [11]:
#add spaces to C and R!
all_tech_df[all_tech_df['tech'] == 'C'] = " C "
all_tech_df[all_tech_df['tech'] == 'R'] = " R "
all_tech_df[all_tech_df['tech'] == 'Go'] = " Go "

In [12]:
#pickle out the simple list
all_tech_df.to_pickle('~/DataScienceJobs/data/tech_dictionary.pkl')

In [13]:
agg_dict={'back_end-technologies':['javascript', 'php', 'ruby', 'java', 'c-net', 'python', 'c_plus_plus', 'c', 'scala', 'golang', 'functional-programming-fp', 'other-programming_languages'],
         'mobile':['ios', 'android', 'cross_platform-hybrid'], 'cyber_security':['cyber_security/arcsight','cyber_security/nessus', 'cyber_security/iso']}

In [14]:
tech_dict['back_end-technologies']=[]
for x in agg_dict['back_end-technologies']:
    tech_dict['back_end-technologies'].append(tech_dict[x])

In [15]:
tech_dict['mobile']=[]
for x in agg_dict['mobile']:
    tech_dict['mobile'].append(tech_dict[x])

In [16]:
for x in agg_dict['cyber_security']:
    [tech_dict['cyber_security']].append(tech_dict[x])

In [17]:
#tech_dict['cyber_security'] = [tech_dict['cyber_security'] + tech_dict[x] for x in agg_dict['cyber_security']]

In [18]:
for i in agg_dict.values():
    for element in i:
        del tech_dict[element]

In [19]:
tech_dict.keys()

dict_keys(['front_end-technologies', 'databases', 'quality_assurance-qa', 'game_development', 'software-infrastructure-devops', 'web_design', 'product_management', 'development_methodologies', 'software_architecture', 'fundamental_programming_concepts', 'programming_paradigms', 'data-science', 'tools', 'roles_in_software_development', 'embedded', 'cloud_computing', 'cyber_security', 'general_terms', 'back_end-technologies', 'mobile'])

In [35]:
for key in tech_dict:
    a = str(tech_dict[key]).replace(']',",").replace('[',",").replace("'","").replace("\\","").split(",")
    b = [x.lstrip().rstrip() for x in a]
    for i in range(len(b)):
        if b[i] == 'C':
            b[i] = ' C '
        elif b[i] == 'R':
            b[i]= ' R '
        elif b[i] == 'Go':
            b[i] = ' Go '
        elif b[i] == 'Vi':
            b[i] = ' Vi '
        else:
            pass 
    c = list(dict.fromkeys(b))
    tech_dict[key] = c
    try:
        tech_dict[key].remove('')
    except:
        pass
    try:
        tech_dict[key].remove('"')
    except:
        pass
    try:
        tech_dict[key].remove('vi')
    except:
        pass

In [36]:
len(b)

185

In [37]:
key

'mobile'

In [38]:
print(i)

184


In [39]:
range(len(b)-1)

range(0, 184)

In [40]:

tech_dict['front_end-technologies']

['Front-end',
 'HTML5',
 'CSS',
 'JavaScript',
 'AJAX',
 'XML',
 'HTML',
 'AMP',
 'Angular',
 'MVC',
 'Angular 2',
 'AngularFire',
 'Firebase',
 'Angular CLI',
 'Atom',
 'Aurelia',
 'Babel',
 'ES6/ES7',
 'ES5',
 'BabylonJS',
 'WebGL',
 'Backbone.js',
 'RESTful API',
 'BEM',
 'Blaze',
 'Meteor',
 'Bootstrap',
 'JavaSсript',
 'Sass',
 'jQuery',
 'Bourbon',
 'Bower',
 'front-end',
 'JS',
 'Brackets',
 'Broccoli',
 'Node.js',
 'Browserify',
 'Brunch',
 'CanvasJS',
 'Android',
 'Chart.js',
 'ClojureScript',
 'Clojure',
 'Compass',
 'Core.js',
 'CSS Modules',
 'CSS Preprocessor',
 'CSS-related',
 'LESS',
 'YAML',
 'Zurb',
 'Cypress',
 'D3.js',
 'DOM',
 'SVG',
 'DHTML',
 'Dojo',
 'UI',
 'API',
 'Dust.js',
 'ECMAScript',
 'ActionScript',
 'Electron',
 'Ember.js',
 'ESLint',
 'ExtJS',
 'MVVM',
 'Flexbox',
 'Flux',
 'React',
 'Foundation',
 'GraphQL',
 'Grunt',
 'npm',
 'GSAP',
 'Gulp',
 'Less',
 'Haml',
 'Rubyxa0on Rails',
 'Handlebars.js',
 'Highcharts',
 'HTML-related',
 'Bootstap',
 'Canvas'

In [33]:
import os
os.getcwd()
os.chdir('..')

In [41]:
# convert to dataframe for pickling
# with open('~/DataScienceJobs/data/broad_tech_dictionary.pickle', 'wb') as handle:
#     pickle.dump(tech_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
pickle.dump(tech_dict, open('Pickles/broad_tech_dictionary.pickle', 'wb'))

In [384]:
#pickle.load(open('data/broad_tech_dictionary.pickle', 'wb'))

{'front_end-technologies': ['',
  'Front-end',
  'HTML5',
  'CSS',
  'JavaScript',
  'AJAX',
  'XML',
  'HTML',
  'AMP',
  'Angular',
  'MVC',
  'Angular 2',
  'AngularFire',
  'Firebase',
  'Angular CLI',
  'Atom',
  'Aurelia',
  'Babel',
  'ES6/ES7',
  'ES5',
  'BabylonJS',
  'WebGL',
  'Backbone.js',
  'RESTful API',
  'BEM',
  'Blaze',
  'Meteor',
  'Bootstrap',
  'JavaSсript',
  'Sass',
  'jQuery',
  'Bourbon',
  'Bower',
  'front-end',
  'JS',
  'Brackets',
  'Broccoli',
  'Node.js',
  'Browserify',
  'Brunch',
  'CanvasJS',
  'Android',
  'Chart.js',
  'ClojureScript',
  'Clojure',
  'Compass',
  'Core.js',
  'CSS Modules',
  'CSS Preprocessor',
  'CSS-related',
  'LESS',
  'YAML',
  'Zurb',
  'Cypress',
  'D3.js',
  'DOM',
  'SVG',
  'DHTML',
  'Dojo',
  'UI',
  'API',
  'Dust.js',
  'ECMAScript',
  'ActionScript',
  'Electron',
  'Ember.js',
  'ESLint',
  'ExtJS',
  'MVVM',
  'Flexbox',
  'Flux',
  'React',
  'Foundation',
  'GraphQL',
  'Grunt',
  'npm',
  'GSAP',
  'Gulp',
 